# Fuzzing with Grammars

In the chapter on ["Mutation-Based Fuzzing"](MutationFuzzer.ipynb), we have seen how to use extra hints – such as sample input files – to speed up test generation.  In this chapter, we take this idea one step further, by providing a _specification_ of the legal inputs to a program.  Specifying inputs via a _grammar_ allows for very systematic and efficient test generation, in particular for complex input formats.  Grammars also serve as the base for configuration fuzzing, API fuzzing, GUI fuzzing, and many more.

**Prerequisites**

* You should know how basic fuzzing works, e.g. from the [Chapter introducing fuzzing](Fuzzer.ipynb).
* Knowledge on [mutation-based fuzzing](MutationFuzzer.ipynb) and [coverage](Coverage.ipynb) is _not_ required yet, but still recommended.

In [ ]:
import fuzzingbook_utils

In [ ]:
import Fuzzer

## Synopsis
<!-- Automatically generated. Do not edit. -->

To [use the code provided in this chapter](Importing.ipynb), write

```python
>>> from fuzzingbook.Grammars import <identifier>
```

and then make use of the following features.


This chapter introduces _grammars_ as a simple means to specify input languages, and to use them for testing programs with syntactically valid inputs.  A grammar is defined as a mapping of nonterminal symbols to lists of alternative expansions, as in the following example:

```python
>>> US_PHONE_GRAMMAR = {
>>>     "<start>": ["<phone-number>"],
>>>     "<phone-number>": ["(<area>)<exchange>-<line>"],
>>>     "<area>": ["<lead-digit><digit><digit>"],
>>>     "<exchange>": ["<lead-digit><digit><digit>"],
>>>     "<line>": ["<digit><digit><digit><digit>"],
>>>     "<lead-digit>": ["2", "3", "4", "5", "6", "7", "8", "9"],
>>>     "<digit>": ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
>>> }
>>> 
>>> assert is_valid_grammar(US_PHONE_GRAMMAR)
```
Nonterminal symbols are enclosed in angle brackets (say, `<digit>`).  To generate an input string from a grammar, a _producer_ starts with the start symbol (`<start>`) and randomly chooses a random expansion for this symbol.  It continues the process until all nonterminal symbols are expanded.  The function `simple_grammar_fuzzer()` does just that:

```python
>>> [simple_grammar_fuzzer(US_PHONE_GRAMMAR) for i in range(5)]
['(291)427-2390',
 '(206)588-5240',
 '(770)592-1561',
 '(650)562-4857',
 '(955)809-9273']
```
In practice, though, instead of `simple_grammar_fuzzer()`, you should use [the `GrammarFuzzer` class](GrammarFuzzer.ipynb) or one of its [coverage-based](GrammarCoverageFuzzer.ipynb), [probabilistic-based](ProbabilisticGrammarFuzzer.ipynb), or [generator-based](GeneratorGrammarFuzzer.ipynb) derivatives; these are more efficient, protect against infinite growth, and provide several additional features.

This chapter also introduces a [grammar toolbox](#A-Grammar-Toolbox) with several helper functions that ease the writing of grammars, such as using shortcut notations for character classes and repetitions, or extending grammars 



## Input Languages

All possible behaviors of a program can be triggered by its input.  "Input" here can be a wide range of possible sources: We are talking about data that is read from files, from the environment, or over the network, data input by the user, or data acquired from interaction with other resources.  The set of all these inputs determines how the program will behave – including its failures.  When testing, it is thus very helpful to think about possible input sources, how to get them under control, and _how to systematically test them_.

For the sake of simplicity, we will assume for now that the program has only one source of inputs; this is the same assumption we have been using in the previous chapters, too.  The set of valid inputs to a program is called a _language_.  Languages range from the simple to the complex: the CSV language denotes the set of valid comma-separated inputs, whereas the Python language denotes the set of valid Python programs.

## Grammars

Grammars are among the most popular (and best understood) formalisms to formally specify input languages.  Using a grammar, one can express a wide range of the properties of an input language.  Grammars are particularly great for expressing the *syntactical structure* of an input, and are the formalism of choice to express nested or recursive inputs.  The grammars we use are so-called *context-free grammars*, one of the easiest and most popular grammar formalisms.

### Rules and Expansions

A grammar consists of a *start symbol* and a set of *expansion rules* (or simply *rules*) which indicate how the start symbol (and other symbols) can be expanded.  As an example, consider the following grammar, denoting a sequence of two digits:

```
<start> ::= <digit><digit>
<digit> ::= 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9
```

To read such a grammar, start with the start symbol (`<start>`).  An expansion rule `<A> ::= <B>` means that the symbol on the left side (`<A>`) can be replaced by the string on the right side (`<B>`).  In the above grammar, `<start>` would be replaced by `<digit><digit>`.

In this string again, `<digit>` would be replaced by the string on the right side of the `<digit>` rule.  The special operator `|` denotes *expansion alternatives* (or simply *alternatives*), meaning that any of the digits can be chosen for an expansion.  Each `<digit>` thus would be expanded into one of the given digits, eventually yielding a string between `00` and `99`.  There are no further expansions for `0` to `9`, so we are all set.

The interesting thing about grammars is that they can be *recursive*. That is, expansions can make use of symbols expanded earlier – which would then be expanded again.  As an example, consider a grammar that describes integers:

```
<start>  ::= <integer>
<integer> ::= <digit> | <digit><integer>
<digit>   ::= 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9
```

Here, a `<integer>` is either a single digit, or a digit followed by another integer.  The number `1234` thus would be represented as a single digit `1`, followed by the integer `234`, which in turn is a digit `2`, followed by the integer `34`.

If we wanted to express that an integer can be preceded by a sign (`+` or `-`), we would write the grammar as

```
<start>   ::= <number>
<number>  ::= <integer> | +<integer> | -<integer>
<integer> ::= <digit> | <digit><integer>
<digit>   ::= 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9
```

These rules formally define the language: Anything that can be derived from the start symbol is part of the language; anything that cannot is not.

### Arithmetic Expressions

Let us expand our grammar to cover full *arithmetic expressions* – a poster child example for a grammar.  We see that an expression (`<expr>`) is either a sum, or a difference, or a term; a term is either a product or a division, or a factor; and a factor is either a number or a parenthesized expression.  Almost all rules can have recursion, and thus allow arbitrary complex expressions such as `(1 + 2) * (3.4 / 5.6 - 789)`.

```
<start>   ::= <expr>
<expr>    ::= <term> + <expr> | <term> - <expr> | <term>
<term>    ::= <term> * <factor> | <term> / <factor> | <factor>
<factor>  ::= +<factor> | -<factor> | (<expr>) | <integer> | <integer>.<integer>
<integer> ::= <digit><integer> | <digit>
<digit>   ::= 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9
```

In such a grammar, if we start with `<start>` and then expand one symbol after another, randomly choosing alternatives, we can quickly produce one valid arithmetic expression after another.  Such *grammar fuzzing* is highly effective as it comes to produce complex inputs, and this is what we will implement in this chapter.

## Representing Grammars in Python

Our first step in building a grammar fuzzer is to find an appropriate format for grammars.  To make the writing of grammars as simple as possible, we use a format that is based on strings and lists.  Our grammars in Python take the format of a _mapping_ between symbol names and expansions, where expansions are _lists_ of alternatives.  A one-rule grammar for digits thus takes the form

In [ ]:
DIGIT_GRAMMAR = {
    "<start>":
        ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
}

whereas the full grammar for arithmetic expressions looks like this:

In [ ]:
EXPR_GRAMMAR = {
    "<start>":
        ["<expr>"],

    "<expr>":
        ["<term> + <expr>", "<term> - <expr>", "<term>"],

    "<term>":
        ["<factor> * <term>", "<factor> / <term>", "<factor>"],

    "<factor>":
        ["+<factor>",
         "-<factor>",
         "(<expr>)",
         "<integer>.<integer>",
         "<integer>"],

    "<integer>":
        ["<digit><integer>", "<digit>"],

    "<digit>":
        ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
}

In the grammar, every symbol can be defined exactly once.  We can access any rule by its symbol...

In [ ]:
EXPR_GRAMMAR["<digit>"]

....and we can check whether a symbol is in the grammar:

In [ ]:
"<identifier>" in EXPR_GRAMMAR

Note that we assume that on the left hand side of a rule (i.e., the key in the mapping) is always a single symbol.  This is the property that gives our grammars the characterization of _context-free_.

## Some Definitions

We assume that the canonical start symbol is `<start>`:

In [ ]:
START_SYMBOL = "<start>"

The handy `nonterminals()` function extracts the list of nonterminal symbols (i.e., anything between `<` and `>`, except spaces) from an expansion.

In [ ]:
import re

In [ ]:
RE_NONTERMINAL = re.compile(r'(<[^<> ]*>)')

In [ ]:
def nonterminals(expansion):
    # In later chapters, we allow expansions to be tuples,
    # with the expansion being the first element
    if isinstance(expansion, tuple):
        expansion = expansion[0]

    return re.findall(RE_NONTERMINAL, expansion)

In [ ]:
assert nonterminals("<term> * <factor>") == ["<term>", "<factor>"]
assert nonterminals("<digit><integer>") == ["<digit>", "<integer>"]
assert nonterminals("1 < 3 > 2") == []
assert nonterminals("1 <3> 2") == ["<3>"]
assert nonterminals("1 + 2") == []
assert nonterminals(("<1>", {'option': 'value'})) == ["<1>"]

Likewise, `is_nonterminal()` checks whether some symbol is a nonterminal:

In [ ]:
def is_nonterminal(s):
    return re.match(RE_NONTERMINAL, s)

In [ ]:
assert is_nonterminal("<abc>")
assert is_nonterminal("<symbol-1>")
assert not is_nonterminal("+")

## A Simple Grammar Fuzzer

Let us now put the above grammars to use.   We will build a very simple grammar fuzzer that starts with a start symbol (`<start>`) and then keeps on expanding it.  To avoid expansion to infinite inputs, we place a limit (`max_nonterminals`) on the number of nonterminals.  Furthermore, to avoid being stuck in a situation where we cannot reduce the number of symbols any further, we also limit the total number of expansion steps.

In [ ]:
import random

In [ ]:
class ExpansionError(Exception):
    pass

In [ ]:
def simple_grammar_fuzzer(grammar, start_symbol=START_SYMBOL,
                          max_nonterminals=10, max_expansion_trials=100,
                          log=False):
    term = start_symbol
    expansion_trials = 0

    while len(nonterminals(term)) > 0:
        symbol_to_expand = random.choice(nonterminals(term))
        expansions = grammar[symbol_to_expand]
        expansion = random.choice(expansions)
        new_term = term.replace(symbol_to_expand, expansion, 1)

        if len(nonterminals(new_term)) < max_nonterminals:
            term = new_term
            if log:
                print("%-40s" % (symbol_to_expand + " -> " + expansion), term)
            expansion_trials = 0
        else:
            expansion_trials += 1
            if expansion_trials >= max_expansion_trials:
                raise ExpansionError("Cannot expand " + repr(term))

    return term

Let us see how this simple grammar fuzzer obtains an arithmetic expression from the start symbol:

In [ ]:
simple_grammar_fuzzer(grammar=EXPR_GRAMMAR, max_nonterminals=3, log=True)

By increasing the limit of nonterminals, we can quickly get much longer productions:

In [ ]:
for i in range(10):
    print(simple_grammar_fuzzer(grammar=EXPR_GRAMMAR, max_nonterminals=5))

Note that this fuzzer is rather inefficient due to the large number of search and replace operations.  On the other hand, the implementation is straightforward and does the job in most cases.  For this chapter, we'll stick to it; in the [next chapter](GrammarFuzzer.ipynb), we'll show how to build a more efficient one.

## Some Grammars

Let us create (and visualize) some more grammars and use them for fuzzing.

### A CGI Grammar

Here's a grammar for `cgi_decode()` introduced in the [chapter on coverage](Coverage.ipynb).

In [ ]:
CGI_GRAMMAR = {
    "<start>":
        ["<string>"],

    "<string>":
        ["<letter>", "<letter><string>"],

    "<letter>":
        ["<plus>", "<percent>", "<other>"],

    "<plus>":
        ["+"],

    "<percent>":
        ["%<hexdigit><hexdigit>"],

    "<hexdigit>":
        ["0", "1", "2", "3", "4", "5", "6", "7",
            "8", "9", "a", "b", "c", "d", "e", "f"],

    "<other>":  # Actually, could be _all_ letters
        ["0", "1", "2", "3", "4", "5", "a", "b", "c", "d", "e", "-", "_"],
}

In contrast to [basic fuzzing](Fuzzer.ipynb) or [mutation-based fuzzing](MutationFuzzer.ipynb), the grammar quickly produces all sorts of combinations:

In [ ]:
for i in range(10):
    print(simple_grammar_fuzzer(grammar=CGI_GRAMMAR, max_nonterminals=10))

### A URL Grammar

The same properties we have seen for CGI input also hold for more complex inputs.  Let us use a grammar to produce a large number of valid URLs:

In [ ]:
URL_GRAMMAR = {
    "<start>":
        ["<url>"],
    "<url>":
        ["<scheme>://<authority><path><query>"],
    "<scheme>":
        ["http", "https", "ftp", "ftps"],
    "<authority>":
        ["<host>", "<host>:<port>", "<userinfo>@<host>", "<userinfo>@<host>:<port>"],
    "<host>":  # Just a few
        ["unibw.de", "www.google.com", "example.com"],
    "<port>":
        ["80", "8080", "<nat>"],
    "<nat>":
        ["<digit>", "<digit><digit>"],
    "<digit>":
        ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"],
    "<userinfo>":  # Just one
        ["user:password"],
    "<path>":  # Just a few
        ["", "/", "/<id>"],
    "<id>":  # Just a few
        ["abc", "def", "x<digit><digit>"],
    "<query>":
        ["", "?<params>"],
    "<params>":
        ["<param>", "<param>&<params>"],
    "<param>":  # Just a few
        ["<id>=<id>", "<id>=<nat>"],
}

Again, within milliseconds, we can produce plenty of valid inputs.

In [ ]:
for i in range(10):
    print(simple_grammar_fuzzer(grammar=URL_GRAMMAR, max_nonterminals=10))

## Grammars as Mutation Seeds

One very useful property of grammars is that they produce mostly valid inputs.  From a syntactical standpoint, the inputs are actually _always_ valid, as they satisfy the constraints of the given grammar.  (Of course, one needs a valid grammar in the first place.)  However, there are also _semantical_ properties that cannot be easily expressed in a grammar.  If, say, for a URL, the port range is supposed to be between 1024 and 2048, this is hard to write in a grammar.  If one has to satisfy more complex constraints, one quickly reaches the limits of what a grammar can express.

One way around this is to attach constraints to grammars, as we will discuss [later in this book](ConstraintFuzzer.ipynb).  Another possibility is to put together the strengths of grammar-based fuzzing and [mutation-based fuzzing](MutationFuzzer.ipynb).  The idea is to use the grammar-generated inputs as *seeds* for further mutation-based fuzzing.  This way, we can explore not only _valid_ inputs, but also check out the _boundaries_ between valid and invalid inputs.  This is particularly interesting as  slightly invalid inputs allow to find parser errors (which are often abundant).  As with fuzzing in general, it is the unexpected which reveals errors in programs.

To use our generated inputs as seeds, we can feed them directly into the mutation fuzzers introduced earlier:

In [ ]:
from MutationFuzzer import MutationFuzzer  # minor dependency

In [ ]:
number_of_seeds = 10
seeds = [
    simple_grammar_fuzzer(
        grammar=URL_GRAMMAR,
        max_nonterminals=10) for i in range(number_of_seeds)]
seeds

In [ ]:
m = MutationFuzzer(seeds)

In [ ]:
[m.fuzz() for i in range(20)]

While the first 10 `fuzz()` calls return the seeded inputs (as designed), the later ones again create arbitrary mutations.  Using `MutationCoverageFuzzer` instead of `MutationFuzzer`, we could again have our search guided by coverage – and thus bring together the best of multiple worlds.

## A Grammar Toolbox

Let us now introduce a few techniques that help us writing grammars.

### Escapes

With `<` and `>` delimiting nonterminals in our grammars, how can we actually express that some input should contain `<` and `>`?  The answer is simple: Just introduce a symbol for them.

In [ ]:
simple_nonterminal_grammar = {
    "<start>": ["<nonterminal>"],
    "<nonterminal>": ["<left-angle><identifier><right-angle>"],
    "<left-angle>": ["<"],
    "<right-angle>": [">"],
    "<identifier>": ["id"]  # for now
}

In `simple_nonterminal_grammar`, neither the expansion for `<left-angle>` nor the expansion for `<right-angle>` can be mistaken as a nonterminal.  Hence, we can produce as many as we want.

### Extending Grammars

In the course of this book, we frequently run into the issue of creating a grammar by _extending_ an existing grammar with new features.  Such an extension is very much like subclassing in object-oriented programming.

To create a new grammar $g'$ from an existing grammar $g$, we first copy $g$ into $g'$, and then go and extend existing rules with new alternatives and/or add new symbols.  Here's an example, extending the above `nonterminal` grammar with a better rule for identifiers:

In [ ]:
import copy

In [ ]:
nonterminal_grammar = copy.deepcopy(simple_nonterminal_grammar)
nonterminal_grammar["<identifier>"] = ["<idchar>", "<identifier><idchar>"]
nonterminal_grammar["<idchar>"] = ['a', 'b', 'c', 'd']  # for now

In [ ]:
nonterminal_grammar

Since such an extension of grammars is a common operation, we introduce a custom function `extend_grammar()` which first copies the given grammar and then updates it from a dictionary, using the Python dictionary `update()` method:

In [ ]:
def extend_grammar(grammar, extension={}):
    new_grammar = copy.deepcopy(grammar)
    new_grammar.update(extension)
    return new_grammar

This call to `extend_grammar()` extends `simple_nonterminal_grammar` to `nonterminal_grammar` just like the "manual" example above:

In [ ]:
nonterminal_grammar = extend_grammar(simple_nonterminal_grammar,
                                     {
                                         "<identifier>": ["<idchar>", "<identifier><idchar>"],
                                         # for now
                                         "<idchar>": ['a', 'b', 'c', 'd']
                                     }
                                     )

### Character Classes

In the above `nonterminal_grammar`, we have enumerated only the first few letters; indeed, enumerating all letters or digits in a grammar manually, as in `<idchar> ::= 'a' | 'b' | 'c' ...` is a bit painful.

However, remember that grammars are part of a program, and can thus also be constructed programmatically.  We introduce a function `srange()` which constructs a list of characters in a string:

In [ ]:
import string

In [ ]:
def srange(characters):
    """Construct a list with all characters in the string"""
    return [c for c in characters]

If we pass it the constant `string.ascii_letters`, which holds all ASCII letters, `srange()` returns a list of all ASCII letters:

In [ ]:
string.ascii_letters

In [ ]:
srange(string.ascii_letters)[:10]

We can use such constants in our grammar to quickly define identifiers:

In [ ]:
nonterminal_grammar = extend_grammar(nonterminal_grammar,
                                     {
                                         "<idchar>": srange(string.ascii_letters) + srange(string.digits) + srange("-_")
                                     }
                                     )

In [ ]:
[simple_grammar_fuzzer(nonterminal_grammar, "<identifier>") for i in range(10)]

The shortcut `crange(start, end)` returns a list of all characters in the ASCII range of `start` to (including) `end`:

In [ ]:
def crange(character_start, character_end):
    return [chr(i)
            for i in range(ord(character_start), ord(character_end) + 1)]

We can use this to express ranges of characters:

In [ ]:
crange('0', '9')

In [ ]:
assert crange('a', 'z') == srange(string.ascii_lowercase)

### Grammar Shortcuts

In the above `nonterminal_grammar`, as in other grammars, we have to express repetitions of characters using _recursion_, that is, by referring to the original definition:

In [ ]:
nonterminal_grammar["<identifier>"]

It could be a bit easier if we simply could state that a nonterminal should be a non-empty sequence of letters – for instance, as in

```
<identifier> = <idchar>+
```

where `+` denotes a non-empty repetition of the symbol it follows.

Operators such as `+` are frequently introduced as handy _shortcuts_ in grammars.  Formally, our grammars come in the so-called [Backus-Naur form](https://en.wikipedia.org/wiki/Backus-Naur_form), or *BNF* for short.  Operators _extend_ BNF to so-called _extended BNF*, or *EBNF* for short:

* The form `<symbol>?` indicates that `<symbol>` is optional – that is, it can occur 0 or 1 times.
* The form `<symbol>+` indicates that `<symbol>` can occur 1 or more times repeatedly.
* The form `<symbol>*` indicates that `<symbol>` can occur 0 or more times.  (In other words, it is an optional repetition.)

To make matters even more interesting, we would like to use _parentheses_ with the above shortcuts.  Thus, `(<foo><bar>)?` indicates that the sequence of `<foo>` and `<bar>` is optional.

Using such operators, we can define the identifier rule in a simpler way.  To this end, let us create a copy of the original grammar and modify the `<identifier>` rule:

In [ ]:
nonterminal_ebnf_grammar = extend_grammar(nonterminal_grammar,
                                          {
                                              "<identifier>": ["<idchar>+"]
                                          }
                                          )

Likewise, we can simplify the expression grammar.  Consider how signs are optional, and how integers can be expressed as sequences of digits.

In [ ]:
EXPR_EBNF_GRAMMAR = {
    "<start>":
        ["<expr>"],

    "<expr>":
        ["<term> + <expr>", "<term> - <expr>", "<term>"],

    "<term>":
        ["<factor> * <term>", "<factor> / <term>", "<factor>"],

    "<factor>":
        ["<sign>?<factor>", "(<expr>)", "<integer>(.<integer>)?"],

    "<sign>":
        ["+", "-"],

    "<integer>":
        ["<digit>+"],

    "<digit>":
        srange(string.digits)
}

Our aim is to convert EBNF grammars such as the ones above into a regular BNF grammar.  This is done by four rules:

1. An expression `(content)op`, where `op` is one of `?`, `+`, `*`, becomes `<new-symbol>op`, with a new rule `<new-symbol> ::= content`.
2. An expression `<symbol>?` becomes `<new-symbol>`, where `<new-symbol> ::= <empty> | <symbol>`.
3. An expression `<symbol>+` becomes `<new-symbol>`, where `<new-symbol> ::= <symbol> | <symbol><new-symbol>`.
4. An expression `<symbol>*` becomes `<new-symbol>`, where `<new-symbol> ::= <empty> | <symbol><new-symbol>`.

Here, `<empty>` expands to the empty string, as in `<empty> ::= `.  (This is also called an *epsilon expansion*.)

If these operators remind you of _regular expressions_, this is not by accident: Actually, any basic regular expression can be converted into a grammar using the above rules (and character classes with `crange()`, as defined above).

Applying these rules on the examples above yields the following results:

* `<idchar>+` becomes `<idchar><new-symbol>` with `<new-symbol> ::= <idchar> | <idchar><new-symbol>`.  
* `<integer>(.<integer>)?` becomes `<integer><new-symbol>` with `<new-symbol> ::= <empty> | .<integer>`.

Let us implement these rules in three steps.

#### Creating New Symbols

First, we need a mechanism to create new symbols.  This is fairly straightforward.

In [ ]:
def new_symbol(grammar, symbol_name="<symbol>"):
    """Return a new symbol for `grammar` based on `symbol_name`"""
    if symbol_name not in grammar:
        return symbol_name

    count = 1
    while True:
        tentative_symbol_name = symbol_name[:-1] + "-" + repr(count) + ">"
        if tentative_symbol_name not in grammar:
            return tentative_symbol_name
        count += 1

In [ ]:
assert new_symbol(EXPR_EBNF_GRAMMAR, '<expr>') == '<expr-1>'

#### Expanding Parenthesized Expressions

Next, we need a means to extract parenthesized expressions from our expansions and expand them according to the rules above.  Let's start with extracting expressions:

In [ ]:
RE_PARENTHESIZED_EXPR = re.compile(r'\([^()]*\)[?+*]')

In [ ]:
def parenthesized_expressions(expansion):
    # In later chapters, we allow expansions to be tuples,
    # with the expansion being the first element
    if isinstance(expansion, tuple):
        expansion = expansion[0]

    return re.findall(RE_PARENTHESIZED_EXPR, expansion)

In [ ]:
assert parenthesized_expressions("(<foo>)* (<foo><bar>)+ (+<foo>)? <integer>(.<integer>)?") == [
    '(<foo>)*', '(<foo><bar>)+', '(+<foo>)?', '(.<integer>)?']

We can now use these to apply rule number 1, above, introducing new symbols for expressions in parentheses.

In [ ]:
def convert_ebnf_parentheses(ebnf_grammar):
    """Convert a grammar in extended BNF to BNF"""
    grammar = extend_grammar(ebnf_grammar)
    for nonterminal in ebnf_grammar:
        expansions = ebnf_grammar[nonterminal]

        for i in range(len(expansions)):
            expansion = expansions[i]

            while True:
                parenthesized_exprs = parenthesized_expressions(expansion)
                if len(parenthesized_exprs) == 0:
                    break

                for expr in parenthesized_exprs:
                    operator = expr[-1:]
                    contents = expr[1:-2]

                    new_sym = new_symbol(grammar)
                    expansion = grammar[nonterminal][i].replace(
                        expr, new_sym + operator, 1)
                    grammar[nonterminal][i] = expansion
                    grammar[new_sym] = [contents]

    return grammar

This does the conversion as sketched above:

In [ ]:
convert_ebnf_parentheses({"<number>": ["<integer>(.<integer>)?"]})

It even works for nested parenthesized expressions:

In [ ]:
convert_ebnf_parentheses({"<foo>": ["((<foo>)?)+"]})

#### Expanding Operators

After expanding parenthesized expressions, we now need to take care of symbols followed by operators (`?`, `*`, `+`).  As with `convert_ebnf_parentheses()`, above, we first extract all symbols followed by an operator.

In [ ]:
RE_EXTENDED_NONTERMINAL = re.compile(r'(<[^<> ]*>[?+*])')

In [ ]:
def extended_nonterminals(expansion):
    # In later chapters, we allow expansions to be tuples,
    # with the expansion being the first element
    if isinstance(expansion, tuple):
        expansion = expansion[0]

    return re.findall(RE_EXTENDED_NONTERMINAL, expansion)

In [ ]:
assert extended_nonterminals(
    "<foo>* <bar>+ <elem>? <none>") == ['<foo>*', '<bar>+', '<elem>?']

Our converter extracts the symbol and the operator, and adds new symbols according to the rules laid out above.

In [ ]:
def convert_ebnf_operators(ebnf_grammar):
    """Convert a grammar in extended BNF to BNF"""
    grammar = extend_grammar(ebnf_grammar)
    for nonterminal in ebnf_grammar:
        expansions = ebnf_grammar[nonterminal]

        for i in range(len(expansions)):
            expansion = expansions[i]
            extended_symbols = extended_nonterminals(expansion)

            for extended_symbol in extended_symbols:
                operator = extended_symbol[-1:]
                original_symbol = extended_symbol[:-1]

                new_sym = new_symbol(grammar, original_symbol)
                grammar[nonterminal][i] = grammar[nonterminal][i].replace(
                    extended_symbol, new_sym, 1)

                if operator == '?':
                    grammar[new_sym] = ["", original_symbol]
                elif operator == '*':
                    grammar[new_sym] = ["", original_symbol + new_sym]
                elif operator == '+':
                    grammar[new_sym] = [
                        original_symbol, original_symbol + new_sym]

    return grammar

In [ ]:
convert_ebnf_operators({"<integer>": ["<digit>+"]})

#### All Together

We can combine the two, first extending parentheses and then operators:

In [ ]:
def convert_ebnf_grammar(ebnf_grammar):
    return convert_ebnf_operators(convert_ebnf_parentheses(ebnf_grammar))

In [ ]:
convert_ebnf_grammar({"<authority>": ["(<userinfo>@)?<host>(:<port>)?"]})

In [ ]:
expr_grammar = convert_ebnf_grammar(EXPR_EBNF_GRAMMAR)
expr_grammar

Success! We have nicely converted the EBNF grammar into BNF.

With character classes and EBNF grammar conversion, we have two powerful tools that make the writing of grammars easier.  We will use these again and again as it comes to working with grammars.

### Grammar Extensions

During the course of this book, we frequently want to specify _additional information_ for grammars, such as [_probabilities_](ProbabilisticGrammarFuzzer.ipynb) or [_constraints_](GeneratorGrammarFuzzer.ipynb).  To support these extensions, as well as possibly others, we define an _annotation_ mechanism.

Our concept for annotating grammars is to add _annotations_ to individual expansions.  To this end, we allow that an expansion cannot only be a string, but also a _pair_ of a string and a set of attributes, as in

```python
    "<expr>":
        [("<term> + <expr>", opts(min_depth=10)),
         ("<term> - <expr>", opts(max_depth=2)),
         "<term>"]
```

Here, the `opts()` function would allow us to express annotations that apply to the individual expansions; in this case, the addition would be annotated with a `min_depth` value of 10, and the subtraction with a `max_depth` value of 2.  The meaning of these annotations is left to the individual algorithms dealing with the grammars; the general idea, though, is that they can be ignored.

Our `opts()` helper function returns a mapping of its arguments to values:

In [ ]:
def opts(**kwargs):
    return kwargs

In [ ]:
opts(min_depth=10)

To deal with both expansion strings and pairs of expansions and annotations, we access the expansion string and the associated annotations via designated helper functions, `exp_string()` and `exp_opts()`:

In [ ]:
def exp_string(expansion):
    """Return the string to be expanded"""
    if isinstance(expansion, str):
        return expansion
    return expansion[0]

In [ ]:
exp_string(("<term> + <expr>", opts(min_depth=10)))

In [ ]:
def exp_opts(expansion):
    """Return the options of an expansion.  If options are not defined, return {}"""
    if isinstance(expansion, str):
        return {}
    return expansion[1]

In [ ]:
def exp_opt(expansion, attribute):
    """Return the given attribution of an expansion.
    If attribute is not defined, return None"""
    return exp_opts(expansion).get(attribute, None)

In [ ]:
exp_opts(("<term> + <expr>", opts(min_depth=10)))

In [ ]:
exp_opt(("<term> - <expr>", opts(max_depth=2)), 'max_depth')

Finally, we define a helper function that sets a particular option:

In [ ]:
def set_opts(grammar, symbol, expansion, opts=None):
    """Set the options of the given expansion of grammar[symbol] to opts"""
    expansions = grammar[symbol]
    for i, exp in enumerate(expansions):
        if exp_string(exp) != exp_string(expansion):
            continue

        new_opts = exp_opts(exp)
        if opts is None or new_opts == {}:
            new_opts = opts
        else:
            for key in opts:
                new_opts[key] = opts[key]
        if new_opts == {}:
            grammar[symbol][i] = exp_string(exp)
        else:
            grammar[symbol][i] = (exp_string(exp), new_opts)
        return

    raise KeyError(
        "no expansion " +
        repr(symbol) +
        " -> " +
        repr(
            exp_string(expansion)))

## Checking Grammars

Since grammars are represented as strings, it is fairly easy to introduce errors.  So let us introduce a helper function that checks a grammar for consistency.

The helper function `is_valid_grammar()` iterates over a grammar to check whether all used symbols are defined, and vice versa, which is very useful for debugging; it also checks whether all symbols are reachable from the start symbol.  You don't have to delve into details here, but as always, it is important to get the input data straight before we make use of it.

In [ ]:
import sys

In [ ]:
def def_used_nonterminals(grammar, start_symbol=START_SYMBOL):
    defined_nonterminals = set()
    used_nonterminals = {start_symbol}

    for defined_nonterminal in grammar:
        defined_nonterminals.add(defined_nonterminal)
        expansions = grammar[defined_nonterminal]
        if not isinstance(expansions, list):
            print(repr(defined_nonterminal) + ": expansion is not a list",
                  file=sys.stderr)
            return None, None

        if len(expansions) == 0:
            print(repr(defined_nonterminal) + ": expansion list empty",
                  file=sys.stderr)
            return None, None

        for expansion in expansions:
            if isinstance(expansion, tuple):
                expansion = expansion[0]
            if not isinstance(expansion, str):
                print(repr(defined_nonterminal) + ": "
                      + repr(expansion) + ": not a string",
                      file=sys.stderr)
                return None, None

            for used_nonterminal in nonterminals(expansion):
                used_nonterminals.add(used_nonterminal)

    return defined_nonterminals, used_nonterminals

In [ ]:
def reachable_nonterminals(grammar, start_symbol=START_SYMBOL):
    reachable = set()

    def _find_reachable_nonterminals(grammar, symbol):
        nonlocal reachable
        reachable.add(symbol)
        for expansion in grammar.get(symbol, []):
            for nonterminal in nonterminals(expansion):
                if nonterminal not in reachable:
                    _find_reachable_nonterminals(grammar, nonterminal)

    _find_reachable_nonterminals(grammar, start_symbol)
    return reachable

In [ ]:
def unreachable_nonterminals(grammar, start_symbol=START_SYMBOL):
    return grammar.keys() - reachable_nonterminals(grammar, start_symbol)

In [ ]:
def opts_used(grammar):
    used_opts = set()
    for symbol in grammar:
        for expansion in grammar[symbol]:
            used_opts |= set(exp_opts(expansion).keys())
    return used_opts

In [ ]:
def is_valid_grammar(grammar, start_symbol=START_SYMBOL, supported_opts=None):
    defined_nonterminals, used_nonterminals = \
        def_used_nonterminals(grammar, start_symbol)
    if defined_nonterminals is None or used_nonterminals is None:
        return False

    # Do not complain about '<start>' being not used,
    # even if start_symbol is different
    if START_SYMBOL in grammar:
        used_nonterminals.add(START_SYMBOL)

    for unused_nonterminal in defined_nonterminals - used_nonterminals:
        print(repr(unused_nonterminal) + ": defined, but not used",
              file=sys.stderr)
    for undefined_nonterminal in used_nonterminals - defined_nonterminals:
        print(repr(undefined_nonterminal) + ": used, but not defined",
              file=sys.stderr)

    # Symbols must be reachable either from <start> or given start symbol
    unreachable = unreachable_nonterminals(grammar, start_symbol)
    msg_start_symbol = start_symbol
    if START_SYMBOL in grammar:
        unreachable = unreachable - \
            reachable_nonterminals(grammar, START_SYMBOL)
        if start_symbol != START_SYMBOL:
            msg_start_symbol += " or " + START_SYMBOL
    for unreachable_nonterminal in unreachable:
        print(repr(unreachable_nonterminal) + ": unreachable from " + msg_start_symbol,
              file=sys.stderr)

    used_but_not_supported_opts = set()
    if supported_opts is not None:
        used_but_not_supported_opts = opts_used(
            grammar).difference(supported_opts)
        for opt in used_but_not_supported_opts:
            print(
                "warning: option " +
                repr(opt) +
                " is not supported",
                file=sys.stderr)

    return used_nonterminals == defined_nonterminals and len(unreachable) == 0

Our grammars defined above pass the test:

In [ ]:
assert is_valid_grammar(EXPR_GRAMMAR)
assert is_valid_grammar(CGI_GRAMMAR)
assert is_valid_grammar(URL_GRAMMAR)

The check can also be applied to EBNF grammars:

In [ ]:
assert is_valid_grammar(EXPR_EBNF_GRAMMAR)

These ones do not pass the test, though:

In [ ]:
assert not is_valid_grammar({"<start>": ["<x>"], "<y>": ["1"]})

In [ ]:
assert not is_valid_grammar({"<start>": "123"})

In [ ]:
assert not is_valid_grammar({"<start>": []})

In [ ]:
assert not is_valid_grammar({"<start>": [1, 2, 3]})

From here on, we will always use `is_valid_grammar()` when defining a grammar.

## Synopsis

This chapter introduces _grammars_ as a simple means to specify input languages, and to use them for testing programs with syntactically valid inputs.  A grammar is defined as a mapping of nonterminal symbols to lists of alternative expansions, as in the following example:

In [ ]:
US_PHONE_GRAMMAR = {
    "<start>": ["<phone-number>"],
    "<phone-number>": ["(<area>)<exchange>-<line>"],
    "<area>": ["<lead-digit><digit><digit>"],
    "<exchange>": ["<lead-digit><digit><digit>"],
    "<line>": ["<digit><digit><digit><digit>"],
    "<lead-digit>": ["2", "3", "4", "5", "6", "7", "8", "9"],
    "<digit>": ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
}

assert is_valid_grammar(US_PHONE_GRAMMAR)

Nonterminal symbols are enclosed in angle brackets (say, `<digit>`).  To generate an input string from a grammar, a _producer_ starts with the start symbol (`<start>`) and randomly chooses a random expansion for this symbol.  It continues the process until all nonterminal symbols are expanded.  The function `simple_grammar_fuzzer()` does just that:

In [ ]:
[simple_grammar_fuzzer(US_PHONE_GRAMMAR) for i in range(5)]

In practice, though, instead of `simple_grammar_fuzzer()`, you should use [the `GrammarFuzzer` class](GrammarFuzzer.ipynb) or one of its [coverage-based](GrammarCoverageFuzzer.ipynb), [probabilistic-based](ProbabilisticGrammarFuzzer.ipynb), or [generator-based](GeneratorGrammarFuzzer.ipynb) derivatives; these are more efficient, protect against infinite growth, and provide several additional features.

This chapter also introduces a [grammar toolbox](#A-Grammar-Toolbox) with several helper functions that ease the writing of grammars, such as using shortcut notations for character classes and repetitions, or extending grammars 

## Lessons Learned

* Grammars are powerful tools to express and produce syntactically valid inputs.
* Inputs produced from grammars can be used as is, or used as seeds for mutation-based fuzzing.
* Grammars can be extended with character classes and operators to make writing easier.

## Next Steps

As they make a great foundation for generating software tests, we use grammars again and again in this book.  As a sneak preview, we can use grammars to [fuzz configurations](ConfigurationFuzzer.ipynb):

```
<options> ::= <option>*
<option> ::= -h | --version | -v | -d | -i | --global-config <filename>
```

We can use grammars for [fuzzing functions and APIs](APIFuzzer.ipynb) and [fuzzing graphical user interfaces](WebFuzzer.ipynb):

```
<call-sequence> ::= <call>*
<call> ::= urlparse(<url>) | urlsplit(<url>)
```

We can assign [probabilities](ProbabilisticGrammarFuzzer.ipynb) and [constraints](GeneratorGrammarFuzzer.ipynb) to individual expansions:

```
<term>: 50% <factor> * <term> |  30% <factor> / <term> | 20% <factor>
<integer>: <digit>+ { <integer> >= 100 }
```

All these extras become especially valuable as we can

1. _infer grammars automatically_, dropping the need to specify them manually, and
2. _guide them towards specific goals_ such as coverage or critical functions;

which we also discuss for all techniques in this book.

To get there, however, we still have bit of homework to do.  In particular, we first have to learn how to 

* [create an efficient grammar fuzzer](GrammarFuzzer.ipynb)

## Exercises

### Exercise 1: A JSON Grammar

Take a look at the [JSON specification](http://www.json.org) and derive a grammar from it:

* Use _character classes_ to express valid characters
* Use EBNF to express repetitions and optional parts
* Assume that
    - a string is a sequence of digits, ASCII letters, punctuation and space characters without quotes or escapes
    - whitespace is just a single space.
* Use `is_valid_grammar()` to ensure the grammar is valid.

Feed the grammar into `simple_grammar_fuzzer()`.  Do you encounter any errors, and why?

**Solution.** This is a fairly straightforward translation:

In [ ]:
CHARACTERS_WITHOUT_QUOTE = (string.digits
                            + string.ascii_letters
                            + string.punctuation.replace('"', '').replace('\\', '')
                            + ' ')

In [ ]:
JSON_EBNF_GRAMMAR = {
    "<start>": ["<json>"],

    "<json>": ["<element>"],

    "<element>": ["<ws><value><ws>"],

    "<value>": ["<object>", "<array>", "<string>", "<number>", "true", "false", "null"],

    "<object>": ["{<ws>}", "{<members>}"],

    "<members>": ["<member>(,<members>)*"],

    "<member>": ["<ws><string><ws>:<element>"],

    "<array>": ["[<ws>]", "[<elements>]"],

    "<elements>": ["<element>(,<elements>)*"],

    "<element>": ["<ws><value><ws>"],

    "<string>": ['"' + "<characters>" + '"'],
    
    "<characters>": ["<character>*"],

    "<character>": srange(CHARACTERS_WITHOUT_QUOTE),

    "<number>": ["<int><frac><exp>"],

    "<int>": ["<digit>", "<onenine><digits>", "-<digits>", "-<onenine><digits>"],

    "<digits>": ["<digit>+"],

    "<digit>": ['0', "<onenine>"],

    "<onenine>": crange('1', '9'),

    "<frac>": ["", ".<digits>"],

    "<exp>": ["", "E<sign><digits>", "e<sign><digits>"],

    "<sign>": ["", '+', '-'],

    # "<ws>": srange(string.whitespace)

    "<ws>": [" "]
}

assert is_valid_grammar(JSON_EBNF_GRAMMAR)

In [ ]:
JSON_GRAMMAR = convert_ebnf_grammar(JSON_EBNF_GRAMMAR)

In [ ]:
from ExpectError import ExpectError

In [ ]:
for i in range(50):
    with ExpectError():
        print(simple_grammar_fuzzer(JSON_GRAMMAR, '<object>'))

We get these errors because `simple_grammar_fuzzer()` first expands to a maximum number of elements, and then is limited because every further expansion would _increase_ the number of nonterminals, even though these may eventually reduce the string length.  This issue is addressed in the [next chapter](GrammarFuzzer.ipynb), introducing a more solid algorithm for producing strings from grammars.

### Exercise 2: Finding Bugs

The name `simple_grammar_fuzzer()` does not come by accident: The way it expands grammars is limited in several ways.  What happens if you apply `simple_grammar_fuzzer()` on `nonterminal_grammar` and `expr_grammar`, as defined above, and why?

**Solution**.  `nonterminal_grammar` does not work because `simple_grammar_fuzzer()` eventually tries to expand the just generated nonterminal:

In [ ]:
from ExpectError import ExpectError, ExpectTimeout

In [ ]:
with ExpectError():
    simple_grammar_fuzzer(nonterminal_grammar, log=True)

For `expr_grammar`, things are even worse, as `simple_grammar_fuzzer()` can start a series of infinite expansions:

In [ ]:
with ExpectTimeout(1):
    for i in range(10):
        print(simple_grammar_fuzzer(expr_grammar))

Both issues are addressed and discussed in the [next chapter](GrammarFuzzer.ipynb), introducing a more solid algorithm for producing strings from grammars.